In [1]:
import pandas as pd

df = pd.read_csv("/content/train_preprocessed.csv")
text = df['comment_text'].values[:100]

In [2]:
import torch
from sonar.inference_pipelines.text import TextToEmbeddingModelPipeline

embedder = TextToEmbeddingModelPipeline(
  encoder="text_sonar_basic_encoder",
  tokenizer="text_sonar_basic_encoder",
  device=torch.device("cuda"),
  dtype=torch.float16,
)

100%|██████████| 2.85G/2.85G [00:12<00:00, 245MB/s]
100%|██████████| 4.63M/4.63M [00:00<00:00, 43.9MB/s]


In [3]:
with torch.inference_mode():
  embeddings = embedder.predict(text, source_lang="eng_Latn")

print(embeddings.shape)

torch.Size([100, 1024])


/usr/local/lib/python3.11/dist-packages/sonar/inference_pipelines/text.py:259: UserWarning: For 3 input tensors for SONAR text encoder, the length was truncated to 514 elements.
  warnings.warn(


In [4]:
from sonar.inference_pipelines.text import EmbeddingToTextModelPipeline

decoder = EmbeddingToTextModelPipeline(
    decoder="text_sonar_basic_decoder",
    tokenizer="text_sonar_basic_encoder",
    device=torch.device("cuda:0"),
    dtype=torch.float16,
)



100%|██████████| 4.21G/4.21G [02:56<00:00, 25.6MB/s]


In [5]:
with torch.inference_mode():
  text_decoded = decoder.predict(embeddings[:2,:], target_lang="eng_Latn")

print("Texto original:", text[0])
print("Texto reconstruído:", text_decoded[0])

Texto original: explanation why the edits made under my username hardcore metallica fan were reverted  they weren t vandalisms  just closure on some gas after i voted at new york dolls fac  and please don t remove the template from the talk page since i m retired now 
Texto reconstruído: Explanations why the editing done under my username hardcore metallica fan was reversed they were vandalism just blocking on some gas after i voted at new york dolls fan and please do not remove the template from the discussion page since i am now retired


# Nova seção

In [6]:
import numpy as np

# Adiciona os embeddings ao dataframe
df_embed = df.iloc[:len(embeddings)].copy()
df_embed['embedding'] = [emb.detach().cpu().numpy() for emb in embeddings]

# Define categorias para calcular
categorias = ['identity_hate', 'insult', 'obscene', 'severe_toxic', 'threat', 'toxic']
vetores_categoria = {}

# Calcula vetores médios
for cat in categorias:
    subset = df_embed[df_embed[cat] == 1]
    if not subset.empty:
        vetor_medio = np.mean(np.stack(subset['embedding'].values), axis=0)
        vetores_categoria[cat] = vetor_medio
        print(f"{cat}: vetor médio calculado com {len(subset)} exemplos.")
    else:
        print(f"{cat}: nenhum exemplo encontrado.")

# Também podemos calcular o vetor médio de comentários 'neutros'
neutros = df_embed[(df_embed[categorias].sum(axis=1) == 0)]
vetor_neutro = np.mean(np.stack(neutros['embedding'].values), axis=0)
vetores_categoria['neutro'] = vetor_neutro
print(f"neutro: vetor médio calculado com {len(neutros)} exemplos.")

identity_hate: vetor médio calculado com 1 exemplos.
insult: vetor médio calculado com 7 exemplos.
obscene: vetor médio calculado com 8 exemplos.
severe_toxic: vetor médio calculado com 2 exemplos.
threat: vetor médio calculado com 1 exemplos.
toxic: vetor médio calculado com 14 exemplos.
neutro: vetor médio calculado com 86 exemplos.


In [9]:
# Selecione um exemplo tóxico para testar (altere o índice se quiser outro)
idx = 50  # exemplo: 5º comentário da base

# Recupera dados
texto_original = df_embed.iloc[idx]['comment_text']
embedding_original = df_embed.iloc[idx]['embedding']
categorias_ativas = [cat for cat in categorias if df_embed.iloc[idx][cat] == 1]

print("Texto original:", texto_original)
print("Categorias tóxicas:", categorias_ativas)

# Aplica desintoxicação sequencial (simples soma dos vetores corretivos)
embedding_editado = embedding_original.copy()

for cat in categorias_ativas:
    vetor_toxico = vetores_categoria[cat]
    embedding_editado = embedding_editado - vetor_toxico + vetores_categoria['neutro']

# Converte para tensor
embedding_tensor = torch.tensor(embedding_editado).unsqueeze(0).to("cuda").to(torch.float16)

# Reconstrói texto
with torch.inference_mode():
    texto_desintoxicado = decoder.predict(embedding_tensor, target_lang="eng_Latn")

print("Texto original:", texto_original)
print("Texto desintoxicado:", texto_desintoxicado)

Texto original:  bi  you said you wanted to talk  at the bottom of the lead section you have written its promoter speculated in that the skyhook concept could be cost competitive with what is realistically thought to be achievable using a space elevator  but the skyhook is not competitive with other rotating tether concepts  in addition  the rotating skyhook is in fact deemed not engineeringly feasible using presently available materials regarding in addition  the rotating skyhook is in fact deemed not engineeringly feasible using presently available materials that statement appears to come from ref on page the full quote is if the mass of the tether alone started to exceed times the mass of the payload  then that was an indication the particular scenario being considered was not engineeringly feasible using presently available materials  although the application might become feasible in the near future as better materials become available with higher tensile strengths at higher operat

Nova seção

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split

X = np.stack(df_embed['embedding'].values)
Y = df_embed[categorias].astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Modelo base binário
base_model = LogisticRegression(max_iter=1000)
multi_model = MultiOutputClassifier(base_model)

# Treina
multi_model.fit(X_train, Y_train)

# Previsão com probabilidades
probs = multi_model.predict_proba([X_test[0]])
pesos = [p[1] if len(p) > 1 else 0.0 for p in probs] # pega a probabilidade da classe 1 para cada categoria
print("Probabilidades para cada categoria:", pesos)

Probabilidades para cada categoria: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [12]:
E = X_test[0]
E_editado = E.copy()

for i, cat in enumerate(categorias):
    E_editado -= pesos[i] * vetores_categoria[cat]

E_editado += sum(pesos) * vetores_categoria['neutro']

In [15]:
# Rconstrói texto editado

E_tensor = torch.tensor(E_editado, dtype=torch.float16).unsqueeze(0)  # shape (1, D)
E_tensor_list = [E_tensor]  # precisa ser uma lista de tensores

with torch.inference_mode():
    texto_desintoxicado = decoder.predict(E_tensor_list, target_lang="eng_Latn")

print("Texto original:", texto_original)
print("Texto desintoxicado:", texto_desintoxicado)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x64 and 1024x1024)